# CЕМИНАР. Кластеризация
---

Папулин С.Ю. (papulin.study@yandex.ru)

### Содержание

- [Генерация данных](#Генерация-данных)
- [Метод k-средних (KMeans)](#Метод-k-средних-(KMeans))
- [Иерархическая кластеризация (Agglomerative Clustering)](#Иерархическая-кластеризация (Agglomerative-Clustering))
- [Кластеризация по плотности (DBSCAN)](#Кластеризация-по-плотности-(DBSCAN))
- [Кластеризация изображений цифр](#Кластеризация-изображений-цифр)
- [Источники](#Источники)

Подключение библиотек:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN

from sklearn import metrics

from sklearn.datasets import make_blobs

%matplotlib inline

In [ ]:
from matplotlib.colors import ListedColormap

In [ ]:
clr_map = ListedColormap(["blue", "red", "green", "yellow", "purple"])

## Генерация данных

In [ ]:
from sklearn import datasets

In [ ]:
NUM_SAMPLES = 500
RANDOM_STATE = 12345

In [ ]:
datums = [
    ("merce", datasets.make_classification(
        n_samples=NUM_SAMPLES, 
        n_features=2, 
        n_redundant=0,
        n_informative=2, 
        n_clusters_per_class=1, 
        n_classes=3, 
        class_sep=2,
        random_state=RANDOM_STATE)
    ),
    ("blobs", datasets.make_blobs(
        n_samples=NUM_SAMPLES, 
        n_features=2, 
        centers=5, 
        cluster_std=1.5, 
        random_state=RANDOM_STATE)
    ),
    ("moons", datasets.make_moons(n_samples=NUM_SAMPLES, noise=0.1)),
    ("circles", datasets.make_circles(n_samples=NUM_SAMPLES, factor=0.1, noise=0.1)),
    ("none", (np.random.rand(NUM_SAMPLES, 2), np.zeros(NUM_SAMPLES)))
]

In [ ]:
plt.figure(figsize=[14, 6])

for indx, (title, data) in enumerate(datums):
    
    X, labels = data
    
    plt.subplot(2,5,indx+1)
    plt.title("{}: initial".format(title))
    plt.scatter(X[:,0], X[:,1])
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)
    
    plt.subplot(2,5,indx+1+5)
    plt.title("{}: true labels".format(title))
    plt.scatter(X[:,0], X[:,1], c=labels, cmap=clr_map)
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)

plt.tight_layout()
plt.show()

## Метод k-средних

<p><a href="http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html">KMeans</a></p>

In [ ]:
X, labels = datums[1][1]

In [ ]:
plt.title("Initial data")
plt.scatter(X[:, 0], X[:, 1])
plt.grid(True)
plt.show()

In [ ]:
# Обучение
model = KMeans(n_clusters=5, 
               max_iter=100, 
               init="random",
               n_init=1,
               random_state=10)
model.fit(X)

# Центроиды кластеров
centroids = model.cluster_centers_ 

# Предсказания кластеров
labels_pred = model.labels_  
# или 
labels_pred = model.predict(X)

print("Inertia:", model.inertia_)
print("Centroids:\n{}".format(centroids))
print("Pred labels (first 5): {}".format(labels_pred[:5]))

In [ ]:
# Отображение предсказанных кластеров
plt.figure(figsize=[6, 4])
plt.title("Number of clusters: 5")
plt.scatter(X[:, 0], X[:, 1], c=labels_pred, cmap=clr_map)
plt.scatter(centroids[:, 0], centroids[:, 1], marker="o",
            s=80, color="white", linewidth=3, edgecolors="black", zorder=5)
plt.grid(True)
plt.show()

Измените значение параметра `n_init` на 10 и посморите, что изменится.

### Итерации

In [ ]:
# Замечание: Дважды запустите ячейку

%matplotlib notebook
from matplotlib.animation import FuncAnimation

In [ ]:
fig, ax = plt.subplots(figsize=[8,6])
data_plot = plt.scatter(X[:,0], X[:,1], cmap=clr_map)
centroid_plot = plt.scatter([], [], 
                            marker="o", s=80, color="white", 
                            linewidth=3, edgecolors="black",  
                            zorder=5)


def init():
    ax.set_title("Initial State")
    ax.set_xlabel("$X_1$")
    ax.set_ylabel("$X_2$")
    ax.grid(True)

def update(frame):

    model = KMeans(n_clusters=5, max_iter=frame, 
                   init=np.asarray([[2,2],[1,1],[3,3], [4,4], [5,5]]), 
                   random_state=RANDOM_STATE, n_init=1)
    
    model.fit(X)

    labels_pred = model.labels_
    centroids = model.cluster_centers_
    
    data_plot.set_array(labels_pred)
    centroid_plot.set_offsets(centroids)
    ax.set_title("Iteration {}".format(frame))


ani = FuncAnimation(fig, update, 
                    frames=np.arange(1, 10),
                    init_func=init, interval=2000,
                    repeat=False,
                    blit=False)
plt.show()

In [ ]:
# Замечание: Дважды запустите ячейку

%matplotlib inline

### Разные наборы данных

In [ ]:
models = [
    KMeans(n_clusters=3, max_iter=300, init="random", random_state=RANDOM_STATE, n_init=1),
    KMeans(n_clusters=5, max_iter=300, init="random", random_state=RANDOM_STATE, n_init=1),
    KMeans(n_clusters=2, max_iter=300, init="random", random_state=RANDOM_STATE, n_init=1),
    KMeans(n_clusters=2, max_iter=300, init="random", random_state=RANDOM_STATE, n_init=1),
    KMeans(n_clusters=2, max_iter=300, init="random", random_state=RANDOM_STATE, n_init=1),
]

In [ ]:
plt.figure(figsize=[14, 9])

for indx, (title, data) in enumerate(datums):
    
    X, labels = data
    models[indx].fit(X)
    labels_pred = models[indx].labels_
    
    
    plt.subplot(3,5,indx+1)
    plt.title("{}: initial".format(title))
    plt.scatter(X[:,0], X[:,1])
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)
    
    plt.subplot(3,5,indx+1+5)
    plt.title("{}: true labels".format(title))
    plt.scatter(X[:,0], X[:,1], c=labels, cmap=clr_map)
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)
    
    plt.subplot(3,5,indx+1+10)
    plt.title("{}: pred labels".format(title))
    plt.scatter(X[:,0], X[:,1], c=labels_pred, cmap=clr_map)
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)

plt.tight_layout()
plt.show()

## Иерархическая кластеризация

<a href="http://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html">AgglomerativeClustering</a>

In [ ]:
models = [
    AgglomerativeClustering(n_clusters=3, affinity="euclidean", linkage="average"),
    AgglomerativeClustering(n_clusters=5, affinity="euclidean", linkage="complete"),
    AgglomerativeClustering(n_clusters=2, affinity="euclidean", linkage="single"),
    AgglomerativeClustering(n_clusters=2, affinity="euclidean", linkage="single"),
    AgglomerativeClustering(n_clusters=2, affinity="euclidean", linkage="average")
]

In [ ]:
plt.figure(figsize=[14, 9])

for indx, (title, data) in enumerate(datums):
    
    X, labels = data
    models[indx].fit(X)
    labels_pred = models[indx].labels_
    
    
    plt.subplot(3,5,indx+1)
    plt.title("{}: initial".format(title))
    plt.scatter(X[:,0], X[:,1])
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)
    
    plt.subplot(3,5,indx+1+5)
    plt.title("{}: true labels".format(title))
    plt.scatter(X[:,0], X[:,1], c=labels, cmap=clr_map)
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)
    
    plt.subplot(3,5,indx+1+10)
    plt.title("{}: pred labels".format(title))
    plt.scatter(X[:,0], X[:,1], c=labels_pred, cmap=clr_map)
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)

plt.tight_layout()
plt.show()

## Кластеризация по плотности

<a href="http://scikit-learn.org/stable/modules/generated/sklearn.cluster.DBSCAN.html">DBSCAN</a>

In [ ]:
models = [
    DBSCAN(min_samples=4, eps=0.5, metric="euclidean"),
    DBSCAN(min_samples=4, eps=0.5, metric="euclidean"),
    DBSCAN(min_samples=10, eps=0.2, metric="euclidean"),
    DBSCAN(min_samples=4, eps=0.3, metric="euclidean"),
    DBSCAN(min_samples=4, eps=0.3, metric="euclidean")
]

In [ ]:
plt.figure(figsize=[14, 9])

for indx, (title, data) in enumerate(datums):
    
    X, labels = data
    models[indx].fit(X)
    labels_pred = models[indx].labels_
    
    
    plt.subplot(3,5,indx+1)
    plt.title("{}: initial".format(title))
    plt.scatter(X[:,0], X[:,1])
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)
    
    plt.subplot(3,5,indx+1+5)
    plt.title("{}: true labels".format(title))
    plt.scatter(X[:,0], X[:,1], c=labels, cmap=clr_map)
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)
    
    plt.subplot(3,5,indx+1+10)
    plt.title("{}: pred labels".format(title))
    plt.scatter(X[:,0], X[:,1], c=labels_pred, cmap=clr_map)
    plt.xlabel("$X_1$")
    plt.ylabel("$X_2$")
    plt.grid(True)

plt.tight_layout()
plt.show()

## Кластеризация изображений цифр

In [ ]:
from sklearn import datasets

In [ ]:
RANDOM_STATE = 12345
NUM_CLUSTERS = 10
NUM_DISPLAY_IMAGES = 10

In [ ]:
# Загрузка исходных данных
digits = datasets.load_digits()

IMAGE_INDX = 3

# Отображение одного изображения
print("Image:")
plt.imshow(digits.images[IMAGE_INDX])
plt.show()
print("Feature matrix:\n", digits.images[IMAGE_INDX])
print("\nTarget value:", digits.target[IMAGE_INDX])

### Обучение

In [ ]:
# Преобразование исходных данных
# Замечание: digits.data уже содержит преобразованные данные
X = digits.images.reshape(len(digits.images), -1)
labels_true = digits.target

# Кластеризация
model = KMeans(n_clusters=NUM_CLUSTERS, n_init=10, random_state=RANDOM_STATE)
model.fit(X)

# Выявленные кластеры
labels_pred = model.labels_

for label in range(NUM_CLUSTERS):

    # Индексы элементов кластера label
    labels_pred__indices = np.asarray(labels_pred==label).nonzero()[0]
    
    # Выбираем случайным образом 10 индексов элементов кластера label
    np.random.seed(RANDOM_STATE)
    labels_pred___indices_ = np.random.choice(labels_pred__indices, NUM_DISPLAY_IMAGES, replace=False)
    
    # Отображения выбранных элементов кластера
    print("Cluster label:", label)
    plt.figure(figsize=[14, 4])
    for i in range(NUM_DISPLAY_IMAGES):
        plt.subplot(1, NUM_DISPLAY_IMAGES, i+1)
        plt.title(labels_pred___indices_[i])
        plt.imshow(digits.images[labels_pred___indices_[i]])
        plt.axis("off")
    plt.show()

### Вывод ошибок

In [ ]:
true_indx = 4
pred_indx = 8

true__indices = np.asarray(labels_true==true_indx)
pred__indices = np.asarray(labels_pred==pred_indx)

mismatched__indices = np.asarray(pred__indices!=true__indices).nonzero()[0]

print("Total errors: {} out of {} samples".format(mismatched__indices.size, labels_true.size))

np.random.seed(RANDOM_STATE)
mismatched__indices_ = np.random.choice(mismatched__indices, NUM_DISPLAY_IMAGES, replace=False)


# Отображения выбранных элементов кластера
plt.figure(figsize=[14, 4])
for i in range(NUM_DISPLAY_IMAGES):
    image_indx = mismatched__indices_[i]
    plt.subplot(1, NUM_DISPLAY_IMAGES, i+1)
    plt.title("{}\nTrue: {}\nPred: {}".format(image_indx, 
                                            labels_true[image_indx],
                                            labels_pred[image_indx]))
    plt.imshow(digits.images[image_indx])
    plt.axis("off")
plt.show()

### Метрики

In [ ]:
from sklearn.metrics.cluster import contingency_matrix

In [ ]:
contingency_matrix(labels_true, labels_pred)

## Источники

[Clustering](http://scikit-learn.org/stable/modules/clustering.html)